In [2]:
import pandas as pd

In [3]:
team_profile_df = pd.read_csv('./data/clean_data/team_profile.csv')
team_ranking_df = pd.read_csv('./data/clean_data/team_rankings.csv')
match_overview_df = pd.read_csv('./data/clean_data/match_overview.csv', sep='\t')
match_statistic_df = pd.read_csv('./data/clean_data/match_statistics.csv', sep='\t')

In [4]:
overview_statistic_df = pd.merge(match_overview_df,match_statistic_df, left_on=['Match ID'], right_on=['match_id'])

In [5]:
overview_statistic_df.drop(columns=['match_id'], inplace=True)
overview_statistic_df

,Match ID,Date,Season,Home Team,Away Team,Home Lineup,Away Lineup,Fulltime Score,Winner,home_total_shots,...,home_offsides,home_fouls,away_total_shots,away_shots_saved,away_corners,away_free_kicks,away_offsides,away_fouls,home_cards,away_cards
0,3050173,2018-08-11,18/19,Newcastle United,Tottenham Hotspur,4-4-1-1,4-3-2-1,1:2,Tottenham Hotspur,2,...,11,1,6,1,5,12,12,0,2,2
1,3050169,2018-08-11,18/19,Fulham FC,Crystal Palace,4-3-3 Attacking,4-4-2,0:2,Crystal Palace,6,...,8,2,9,6,5,10,11,3,1,2
2,3050168,2018-08-11,18/19,AFC Bournemouth,Cardiff City,4-4-2 double 6,4-1-4-1,2:0,AFC Bournemouth,3,...,12,0,1,0,4,12,8,2,1,1
3,3050176,2018-08-11,18/19,Wolverhampton Wanderers,Everton FC,3-4-3,4-2-3-1,2:2,Draw,4,...,7,2,5,2,6,9,7,3,0,3
4,3050172,2018-08-10,18/19,Manchester United,Leicester City,4-1-4-1,4-4-2 double 6,2:1,Manchester United,6,...,11,4,4,4,5,15,7,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3474,4095461,2024-05-19,23/24,Sheffield United,Tottenham Hotspur,5-3-2,4-2-3-1,0:3,Tottenham Hotspur,6,...,11,6,18,1,6,11,9,0,2,0
3475,4095443,2024-05-11,23/24,Everton FC,Sheffield United,4-4-1-1,3-5-2 flat,1:0,Everton FC,15,...,6,3,13,6,6,6,11,0,1,2
3476,4095459,2024-05-19,23/24,Luton Town,Fulham FC,3-4-3,4-2-3-1,2:4,Fulham FC,15,...,15,8,15,4,4,14,20,1,5,4
3477,4095452,2024-05-19,23/24,Arsenal FC,Everton FC,4-3-3 Attacking,4-4-1-1,2:1,Arsenal FC,26,...,8,1,5,4,1,8,11,4,4,3


## Raking total shots col

In [6]:
home_shots = overview_statistic_df[['Season', 'Home Team', 'home_total_shots']].rename(
    columns={'Home Team': 'Club', 'home_total_shots': 'total_shots'})
away_shots = overview_statistic_df[['Season', 'Away Team', 'away_total_shots']].rename(
    columns={'Away Team': 'Club', 'away_total_shots': 'total_shots'})

all_shots = pd.concat([home_shots, away_shots], ignore_index=True)
team_shots_by_season = all_shots.groupby(['Season', 'Club']).sum().reset_index()

team_shots_by_season = team_shots_by_season.sort_values(by=['Season', 'total_shots'], ascending=[True, False])
team_shots_by_season['ranking_shots'] = team_shots_by_season.groupby('Season')['total_shots'].rank(ascending=False, method='dense').astype(int)

team_shots_by_season

,Season,Club,total_shots,ranking_shots
9,14/15,Manchester City,222,1
0,14/15,Arsenal FC,218,2
3,14/15,Chelsea FC,210,3
8,14/15,Liverpool FC,187,4
17,14/15,Tottenham Hotspur,173,5
...,...,...,...,...
198,23/24,West Ham United,416,15
187,23/24,Crystal Palace,415,16
191,23/24,Luton Town,394,17
199,23/24,Wolverhampton Wanderers,388,18


In [7]:
final_df = pd.merge(team_profile_df, team_shots_by_season, left_on=['Season', 'Club'], right_on=['Season', 'Club']).drop(columns=['total_shots'])
final_df

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots
0,14/15,Chelsea FC,38,25.4,28,0.358592,3
1,14/15,Manchester United,43,25.4,26,0.321752,6
2,14/15,Manchester City,34,27.1,27,0.281147,1
3,14/15,Arsenal FC,42,24.5,29,0.257345,2
4,14/15,Liverpool FC,35,24.8,21,0.193470,4
...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11
196,23/24,Fulham FC,35,26.9,31,0.235745,9
197,23/24,Burnley FC,39,25.3,30,0.156972,14
198,23/24,Sheffield United,45,24.7,26,0.070979,19


## Raking shot saved col

In [8]:
home_shots_saved = overview_statistic_df[['Season', 'Home Team', 'home_shots_saved']].rename(
    columns={'Home Team': 'Club', 'home_shots_saved': 'total_shots_saved'})
away_shots_saved = overview_statistic_df[['Season', 'Away Team', 'away_shots_saved']].rename(
    columns={'Away Team': 'Club', 'away_shots_saved': 'total_shots_saved'})

all_shots_saved = pd.concat([home_shots_saved, away_shots_saved], ignore_index=True)
team_shots_saved_by_season = all_shots_saved.groupby(['Season', 'Club']).sum().reset_index()

team_shots_saved_by_season = team_shots_saved_by_season.sort_values(by=['Season', 'total_shots_saved'], ascending=[True, False])
team_shots_saved_by_season['ranking_shots_saved'] = team_shots_saved_by_season.groupby('Season')['total_shots_saved'].rank(ascending=False, method='dense').astype(int)

team_shots_saved_by_season

,Season,Club,total_shots_saved,ranking_shots_saved
15,14/15,Sunderland AFC,150,1
16,14/15,Swansea City,148,2
2,14/15,Burnley FC,133,3
18,14/15,West Bromwich Albion,132,4
19,14/15,West Ham United,124,5
...,...,...,...,...
195,23/24,Nottingham Forest,102,15
184,23/24,Brighton & Hove Albion,98,16
187,23/24,Crystal Palace,93,17
192,23/24,Manchester City,72,18


In [9]:
final_df = pd.merge(final_df, team_shots_saved_by_season, left_on=['Season', 'Club'], right_on=['Season', 'Club']).drop(columns=['total_shots_saved'])
final_df

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots,ranking_shots_saved
0,14/15,Chelsea FC,38,25.4,28,0.358592,3,15
1,14/15,Manchester United,43,25.4,26,0.321752,6,13
2,14/15,Manchester City,34,27.1,27,0.281147,1,17
3,14/15,Arsenal FC,42,24.5,29,0.257345,2,11
4,14/15,Liverpool FC,35,24.8,21,0.193470,4,14
...,...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11,10
196,23/24,Fulham FC,35,26.9,31,0.235745,9,6
197,23/24,Burnley FC,39,25.3,30,0.156972,14,3
198,23/24,Sheffield United,45,24.7,26,0.070979,19,2


## Raking corners col

In [10]:
home_corners = overview_statistic_df[['Season', 'Home Team', 'home_corners']].rename(
    columns={'Home Team': 'Club', 'home_corners': 'total_corners'})
away_corners = overview_statistic_df[['Season', 'Away Team', 'away_corners']].rename(
    columns={'Away Team': 'Club', 'away_corners': 'total_corners'})

all_corners = pd.concat([home_corners, away_corners], ignore_index=True)
team_corners_by_season = all_corners.groupby(['Season', 'Club']).sum().reset_index()

team_corners_by_season  = team_corners_by_season.sort_values(by=['Season', 'total_corners'], ascending=[True, False])
team_corners_by_season['ranking_corners'] = team_corners_by_season.groupby('Season')['total_corners'].rank(ascending=False, method='dense').astype(int)

team_corners_by_season 

,Season,Club,total_corners,ranking_corners
9,14/15,Manchester City,257,1
0,14/15,Arsenal FC,245,2
19,14/15,West Ham United,230,3
11,14/15,Newcastle United,217,4
17,14/15,Tottenham Hotspur,210,5
...,...,...,...,...
187,23/24,Crystal Palace,166,14
198,23/24,West Ham United,154,15
195,23/24,Nottingham Forest,148,16
199,23/24,Wolverhampton Wanderers,145,17


In [11]:
final_df = pd.merge(final_df, team_corners_by_season, left_on=['Season', 'Club'], right_on=['Season', 'Club']).drop(columns=['total_corners'])
final_df

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots,ranking_shots_saved,ranking_corners
0,14/15,Chelsea FC,38,25.4,28,0.358592,3,15,6
1,14/15,Manchester United,43,25.4,26,0.321752,6,13,9
2,14/15,Manchester City,34,27.1,27,0.281147,1,17,1
3,14/15,Arsenal FC,42,24.5,29,0.257345,2,11,2
4,14/15,Liverpool FC,35,24.8,21,0.193470,4,14,11
...,...,...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11,10,13
196,23/24,Fulham FC,35,26.9,31,0.235745,9,6,5
197,23/24,Burnley FC,39,25.3,30,0.156972,14,3,13
198,23/24,Sheffield United,45,24.7,26,0.070979,19,2,18


## Ranking free kicks col

In [12]:
home_free_kicks = overview_statistic_df[['Season', 'Home Team', 'home_free_kicks']].rename(
    columns={'Home Team': 'Club', 'home_free_kicks': 'total_free_kicks'})
away_free_kicks = overview_statistic_df[['Season', 'Away Team', 'away_free_kicks']].rename(
    columns={'Away Team': 'Club', 'away_free_kicks': 'total_free_kicks'})

all_free_kicks = pd.concat([home_free_kicks, away_free_kicks], ignore_index=True)
team_free_kicks_by_season = all_free_kicks.groupby(['Season', 'Club']).sum().reset_index()

team_free_kicks_by_season  = team_free_kicks_by_season.sort_values(by=['Season', 'total_free_kicks'], ascending=[True, False])
team_free_kicks_by_season['ranking_free_kicks'] = team_free_kicks_by_season.groupby('Season')['total_free_kicks'].rank(ascending=False, method='dense').astype(int)

team_free_kicks_by_season

,Season,Club,total_free_kicks,ranking_free_kicks
11,14/15,Newcastle United,539,1
1,14/15,Aston Villa,493,2
8,14/15,Liverpool FC,490,3
16,14/15,Swansea City,486,4
12,14/15,Queens Park Rangers,477,5
...,...,...,...,...
188,23/24,Everton FC,340,15
185,23/24,Burnley FC,338,16
180,23/24,AFC Bournemouth,331,17
193,23/24,Manchester United,313,18


In [13]:
final_df = pd.merge(final_df, team_free_kicks_by_season, left_on=['Season', 'Club'], right_on=['Season', 'Club']).drop(columns=['total_free_kicks'])
final_df

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots,ranking_shots_saved,ranking_corners,ranking_free_kicks
0,14/15,Chelsea FC,38,25.4,28,0.358592,3,15,6,10
1,14/15,Manchester United,43,25.4,26,0.321752,6,13,9,11
2,14/15,Manchester City,34,27.1,27,0.281147,1,17,1,16
3,14/15,Arsenal FC,42,24.5,29,0.257345,2,11,2,7
4,14/15,Liverpool FC,35,24.8,21,0.193470,4,14,11,3
...,...,...,...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11,10,13,15
196,23/24,Fulham FC,35,26.9,31,0.235745,9,6,5,13
197,23/24,Burnley FC,39,25.3,30,0.156972,14,3,13,16
198,23/24,Sheffield United,45,24.7,26,0.070979,19,2,18,19


## Ranking offsides col

In [14]:
home_offsides = overview_statistic_df[['Season', 'Home Team', 'home_offsides']].rename(
    columns={'Home Team': 'Club', 'home_offsides': 'total_offsides'})
away_offsides = overview_statistic_df[['Season', 'Away Team', 'away_offsides']].rename(
    columns={'Away Team': 'Club', 'away_offsides': 'total_offsides'})

all_offsides = pd.concat([home_offsides, away_offsides], ignore_index=True)
team_offsides_by_season = all_offsides.groupby(['Season', 'Club']).sum().reset_index()

team_offsides_by_season  = team_offsides_by_season.sort_values(by=['Season', 'total_offsides'], ascending=[True, False])
team_offsides_by_season['ranking_offsides'] = team_offsides_by_season.groupby('Season')['total_offsides'].rank(ascending=False, method='dense').astype(int)

team_offsides_by_season

,Season,Club,total_offsides,ranking_offsides
14,14/15,Stoke City,465,1
4,14/15,Crystal Palace,460,2
7,14/15,Leicester City,426,3
13,14/15,Southampton FC,424,4
11,14/15,Newcastle United,418,5
...,...,...,...,...
189,23/24,Fulham FC,368,15
193,23/24,Manchester United,364,16
194,23/24,Newcastle United,361,17
198,23/24,West Ham United,360,18


In [15]:
final_df = pd.merge(final_df, team_offsides_by_season, left_on=['Season', 'Club'], right_on=['Season', 'Club']).drop(columns=['total_offsides'])
final_df

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots,ranking_shots_saved,ranking_corners,ranking_free_kicks,ranking_offsides
0,14/15,Chelsea FC,38,25.4,28,0.358592,3,15,6,10,19
1,14/15,Manchester United,43,25.4,26,0.321752,6,13,9,11,9
2,14/15,Manchester City,34,27.1,27,0.281147,1,17,1,16,6
3,14/15,Arsenal FC,42,24.5,29,0.257345,2,11,2,7,17
4,14/15,Liverpool FC,35,24.8,21,0.193470,4,14,11,3,16
...,...,...,...,...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11,10,13,15,4
196,23/24,Fulham FC,35,26.9,31,0.235745,9,6,5,13,15
197,23/24,Burnley FC,39,25.3,30,0.156972,14,3,13,16,10
198,23/24,Sheffield United,45,24.7,26,0.070979,19,2,18,19,9


## Ranking fouls col

In [16]:
home_fouls = overview_statistic_df[['Season', 'Home Team', 'home_fouls']].rename(
    columns={'Home Team': 'Club', 'home_fouls': 'total_fouls'})
away_fouls = overview_statistic_df[['Season', 'Away Team', 'away_fouls']].rename(
    columns={'Away Team': 'Club', 'away_fouls': 'total_fouls'})

all_fouls = pd.concat([home_fouls, away_fouls], ignore_index=True)
team_fouls_by_season = all_fouls.groupby(['Season', 'Club']).sum().reset_index()

team_fouls_by_season  = team_fouls_by_season.sort_values(by=['Season', 'total_fouls'], ascending=[True, False])
team_fouls_by_season['ranking_fouls'] = team_fouls_by_season.groupby('Season')['total_fouls'].rank(ascending=False, method='dense').astype(int)

team_fouls_by_season

,Season,Club,total_fouls,ranking_fouls
13,14/15,Southampton FC,80,1
19,14/15,West Ham United,79,2
10,14/15,Manchester United,77,3
17,14/15,Tottenham Hotspur,73,4
5,14/15,Everton FC,69,5
...,...,...,...,...
194,23/24,Newcastle United,57,15
189,23/24,Fulham FC,49,16
180,23/24,AFC Bournemouth,45,17
182,23/24,Aston Villa,45,17


In [17]:
final = pd.merge(final_df, team_fouls_by_season, left_on=['Season', 'Club'], right_on=['Season', 'Club']).drop(columns=['total_fouls'])
final

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots,ranking_shots_saved,ranking_corners,ranking_free_kicks,ranking_offsides,ranking_fouls
0,14/15,Chelsea FC,38,25.4,28,0.358592,3,15,6,10,19,9
1,14/15,Manchester United,43,25.4,26,0.321752,6,13,9,11,9,3
2,14/15,Manchester City,34,27.1,27,0.281147,1,17,1,16,6,5
3,14/15,Arsenal FC,42,24.5,29,0.257345,2,11,2,7,17,13
4,14/15,Liverpool FC,35,24.8,21,0.193470,4,14,11,3,16,6
...,...,...,...,...,...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11,10,13,15,4,2
196,23/24,Fulham FC,35,26.9,31,0.235745,9,6,5,13,15,16
197,23/24,Burnley FC,39,25.3,30,0.156972,14,3,13,16,10,8
198,23/24,Sheffield United,45,24.7,26,0.070979,19,2,18,19,9,14


## Add ranking col

In [18]:
final_df = pd.merge(final_df, team_ranking_df[['Season', 'Team', 'Ranking']], left_on=['Season', 'Club'], right_on=['Season', 'Team']).drop(columns=['Team'])
final_df

,Season,Club,Squad,Average age,Foreigners,Total market value,ranking_shots,ranking_shots_saved,ranking_corners,ranking_free_kicks,ranking_offsides,Ranking
0,14/15,Chelsea FC,38,25.4,28,0.358592,3,15,6,10,19,1
1,14/15,Manchester United,43,25.4,26,0.321752,6,13,9,11,9,4
2,14/15,Manchester City,34,27.1,27,0.281147,1,17,1,16,6,2
3,14/15,Arsenal FC,42,24.5,29,0.257345,2,11,2,7,17,3
4,14/15,Liverpool FC,35,24.8,21,0.193470,4,14,11,3,16,6
...,...,...,...,...,...,...,...,...,...,...,...,...
195,23/24,Everton FC,34,26.8,16,0.241003,11,10,13,15,4,15
196,23/24,Fulham FC,35,26.9,31,0.235745,9,6,5,13,15,13
197,23/24,Burnley FC,39,25.3,30,0.156972,14,3,13,16,10,19
198,23/24,Sheffield United,45,24.7,26,0.070979,19,2,18,19,9,20


In [23]:
final_df.to_csv('./data/clean_data/ranking_data.csv',sep='\t', index=False)